- Due Thursday 11/12 no later than 9:00 a.m., send email to datascience@codeup.com
- Submit link to GitHub notebook that asks and answers questions - document the work you do to justify findings
- Compose an email with the answers to the questions/your findings, and in the email, include the link to your notebook in GitHub and attach your slide. 
- You will not present this, so be sure that the details you need your need your leader to convey/understand are clearly communicated in the email. 
- Slide should be like an exec. Summary and be in form to present. 
- Continue using best practices of acquire.py, prepare.py, etc. 
- No modeling to be done, and no need to split the data into train/validate/test. 
- alumni.codeup.com has info about cohorts/dates/names

- 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
- 2. Is there a cohort that referred to a lesson significantly more that other cohorts seemed to gloss over? 
- 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students? 
- 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses? Any odd user-agents? 
- 5. At some point in the last year, ability for students and alumni to cross-access curriculum (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before? 
- 6. What topics are grads continuing to reference after graduation and into their jobs (for each program)? 
- 7. Which lessons are least accessed? 
- 8. Anything else I should be aware of? 

In [1]:
import numpy as np
import pandas as pd
import math
from sklearn import metrics

from scipy.stats import entropy

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates #to format dates on our plots
%matplotlib inline
import seaborn as sns

In [2]:
df = pd.read_csv('curriculum.txt',
                engine='python',
                 header=None,
                 index_col=False,
                 sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
                 na_values='"-"',)

In [3]:
df.head()

,0,1,2,3,4,5
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [4]:
df.columns = ['date', 'time', 'page_viewed', 'user_id', 'cohort_id', 'ip']

In [5]:
df.head()

,date,time,page_viewed,user_id,cohort_id,ip
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719459 entries, 0 to 719458
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   date         719459 non-null  object 
 1   time         719459 non-null  object 
 2   page_viewed  719458 non-null  object 
 3   user_id      719459 non-null  int64  
 4   cohort_id    674619 non-null  float64
 5   ip           719459 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 32.9+ MB


In [7]:
no_cohort_df = df[df['cohort_id'].isnull()]

In [8]:
no_cohort_df.head()

,date,time,page_viewed,user_id,cohort_id,ip
411,2018-01-26,16:46:16,/,48,NaN,97.105.19.61
412,2018-01-26,16:46:24,spring/extra-features/form-validation,48,NaN,97.105.19.61
425,2018-01-26,17:54:24,/,48,NaN,97.105.19.61
435,2018-01-26,18:32:03,/,48,NaN,97.105.19.61
436,2018-01-26,18:32:17,mysql/relationships/joins,48,NaN,97.105.19.61


**Drop rows with null values and turn**

In [9]:
df.dropna(inplace=True)
# df.cohort_id = df.cohort_id.astype('int')
# df['date'] = df.date + " " + df.time
# df.drop(columns=('time'), inplace=True)
# df.date = pd.to_datetime(df.date)
# df = df.set_index('date')

In [10]:
df.head()

,date,time,page_viewed,user_id,cohort_id,ip
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [11]:
cohort = pd.read_csv('cohort_name.csv')

In [12]:
cohort.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,cohort_id,name,start_date,end_date,program_id,NaN
1,NaN,1,Arches,2014-02-04,2014-04-22,1,NaN
2,NaN,2,Badlands,2014-06-04,2014-08-22,1,NaN
3,NaN,3,Carlsbad,2014-09-04,2014-11-05,1,NaN
4,NaN,4,Denali,2014-10-20,2015-01-18,1,NaN


In [46]:
cohort[cohort.name == 'Ada']

,cohort_id,name,start_date,end_date
30,30,Ada,2019-02-04,2019-06-16


In [13]:
cohort.columns = cohort.iloc[0]

In [14]:
cohort.head()

,NaN,cohort_id,name,start_date,end_date,program_id,NaN
0,NaN,cohort_id,name,start_date,end_date,program_id,NaN
1,NaN,1,Arches,2014-02-04,2014-04-22,1,NaN
2,NaN,2,Badlands,2014-06-04,2014-08-22,1,NaN
3,NaN,3,Carlsbad,2014-09-04,2014-11-05,1,NaN
4,NaN,4,Denali,2014-10-20,2015-01-18,1,NaN


In [15]:
cohort = cohort.iloc[1:]

In [16]:
cohort.head()

,NaN,cohort_id,name,start_date,end_date,program_id,NaN
1,NaN,1,Arches,2014-02-04,2014-04-22,1,NaN
2,NaN,2,Badlands,2014-06-04,2014-08-22,1,NaN
3,NaN,3,Carlsbad,2014-09-04,2014-11-05,1,NaN
4,NaN,4,Denali,2014-10-20,2015-01-18,1,NaN
5,NaN,5,Everglades,2014-11-18,2015-02-24,1,NaN


In [17]:
cohort = cohort[['cohort_id', 'name', 'start_date', 'end_date']]

In [18]:
cohort.head()

,cohort_id,name,start_date,end_date
1,1,Arches,2014-02-04,2014-04-22
2,2,Badlands,2014-06-04,2014-08-22
3,3,Carlsbad,2014-09-04,2014-11-05
4,4,Denali,2014-10-20,2015-01-18
5,5,Everglades,2014-11-18,2015-02-24


In [19]:
cohort.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 1 to 46
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   cohort_id   46 non-null     object
 1   name        46 non-null     object
 2   start_date  46 non-null     object
 3   end_date    46 non-null     object
dtypes: object(4)
memory usage: 1.6+ KB


In [20]:
cohort.cohort_id = cohort.cohort_id.astype('int')


In [49]:
df.cohort_id.value_counts()

28.0    60315
33.0    40168
29.0    37548
53.0    36047
24.0    35624
57.0    32447
56.0    31670
22.0    30328
51.0    29688
58.0    28354
32.0    28333
23.0    28329
26.0    27637
52.0    27518
25.0    25427
31.0    25253
34.0    25181
59.0    22425
27.0    20447
55.0    20410
61.0    11774
14.0     9495
1.0      8884
62.0     8718
21.0     7444
17.0     4925
13.0     2733
18.0     2005
8.0      1712
19.0     1165
16.0      743
15.0      691
7.0       495
12.0      302
11.0      208
2.0        93
6.0        72
9.0         5
4.0         4
5.0         1
Name: cohort_id, dtype: int64

In [21]:
result = pd.merge(df, cohort, on='cohort_id')

In [22]:
result.head()

,date,time,page_viewed,user_id,cohort_id,ip,name,start_date,end_date
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06
4,2018-01-26,10:40:15,javascript-i/functions,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06


In [23]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 674618 entries, 0 to 674617
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   date         674618 non-null  object 
 1   time         674618 non-null  object 
 2   page_viewed  674618 non-null  object 
 3   user_id      674618 non-null  int64  
 4   cohort_id    674618 non-null  float64
 5   ip           674618 non-null  object 
 6   name         674618 non-null  object 
 7   start_date   674618 non-null  object 
 8   end_date     674618 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 51.5+ MB


In [24]:
result.cohort_id = result.cohort_id.astype('int')
result['date'] = result.date + " " + result.time
result.drop(columns=('time'), inplace=True)
result.date = pd.to_datetime(result.date)
result = result.set_index('date')

In [25]:
result.head()

,page_viewed,user_id,cohort_id,ip,name,start_date,end_date
date,,,,,,,
2018-01-26 09:55:03,/,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06
2018-01-26 09:56:02,java-ii,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06
2018-01-26 09:56:06,slides/object_oriented_programming,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06
2018-01-26 10:40:15,javascript-i/functions,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06


In [27]:
result.name.value_counts()

Staff         60315
Ceres         40168
Zion          37548
Fortuna       36047
Voyageurs     35624
Ganymede      32447
Apex          31670
Teddy         30328
Deimos        29688
Hyperion      28354
Betelgeuse    28333
Ulysses       28329
Xanadu        27637
Europa        27518
Wrangell      25427
Andromeda     25253
Bayes         25181
Darden        22425
Yosemite      20447
Curie         20410
Bash          11774
Lassen         9495
Arches         8884
Jupiter        8718
Sequoia        7444
Olympic        4925
Kings          2733
Pinnacles      2005
Hampton        1712
Quincy         1165
Niagara         743
Mammoth         691
Glacier         495
Joshua          302
Ike             208
Badlands         93
Franklin         72
Apollo            5
Denali            4
Everglades        1
Name: name, dtype: int64

In [26]:
pd.crosstab(result.name, result.page_viewed)

page_viewed,%20https://github.com/RaulCPena,",%20https://github.com/RaulCPena",.git,.gitignore,.well-known/assetlinks.json,/,00_,00_index,01_intro,02_listing_files,...,web-design/ui/typography,web-design/ui/visuals,web-design/ux,web-design/ux/layout,web-design/ux/layout/.json,web-design/ux/purpose,web-dev-day-two,working-with-time-series-data,wp-admin,wp-login
name,,,,,,,,,,,,,,,,,,,,,
Andromeda,0,0,0,0,0,1156,0,0,0,0,...,9,11,0,6,0,8,0,0,0,0
Apex,0,0,0,0,0,1244,0,0,0,0,...,20,19,0,17,0,22,0,0,0,0
Apollo,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Arches,0,0,0,0,0,622,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Badlands,0,0,0,0,0,17,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bash,0,0,0,0,0,532,0,0,0,0,...,0,1,0,1,0,0,2,0,0,0
Bayes,0,0,0,0,0,1842,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Betelgeuse,0,0,0,0,0,868,0,0,0,0,...,26,30,0,38,0,32,0,0,0,0
Ceres,0,0,0,0,0,1620,0,0,0,0,...,33,34,0,34,1,28,0,0,0,0


In [41]:
data_science = result[(result.name == 'Curie') | (result.name == 'Bayes') | (result.name == 'Ada') | (result.name == 'Darden')]

In [42]:
data_science.head()

,page_viewed,user_id,cohort_id,ip,name,start_date,end_date
date,,,,,,,
2019-08-20 09:39:58,/,466,34,97.105.19.58,Bayes,2019-08-19,2020-01-30
2019-08-20 09:39:59,/,467,34,97.105.19.58,Bayes,2019-08-19,2020-01-30
2019-08-20 09:39:59,/,468,34,97.105.19.58,Bayes,2019-08-19,2020-01-30
2019-08-20 09:40:02,/,469,34,97.105.19.58,Bayes,2019-08-19,2020-01-30
2019-08-20 09:40:08,/,470,34,97.105.19.58,Bayes,2019-08-19,2020-01-30


In [30]:
data_science.shape

(68016, 7)

In [35]:
web_dev = result[(result.name != 'Curie') & (result.name != 'Bayes') & (result.name != 'Ada') & (result.name != 'Darden')]

In [36]:
web_dev.shape

(606602, 7)

In [37]:
web_dev.name.value_counts()

Staff         60315
Ceres         40168
Zion          37548
Fortuna       36047
Voyageurs     35624
Ganymede      32447
Apex          31670
Teddy         30328
Deimos        29688
Hyperion      28354
Betelgeuse    28333
Ulysses       28329
Xanadu        27637
Europa        27518
Wrangell      25427
Andromeda     25253
Yosemite      20447
Bash          11774
Lassen         9495
Arches         8884
Jupiter        8718
Sequoia        7444
Olympic        4925
Kings          2733
Pinnacles      2005
Hampton        1712
Quincy         1165
Niagara         743
Mammoth         691
Glacier         495
Joshua          302
Ike             208
Badlands         93
Franklin         72
Apollo            5
Denali            4
Everglades        1
Name: name, dtype: int64

In [50]:
pd.crosstab(data_science.name, data_science.page_viewed)

In [43]:
data_science.name.value_counts()

Bayes     25181
Darden    22425
Curie     20410
Name: name, dtype: int64

In [45]:
result[result.name == 'Ada']

,page_viewed,user_id,cohort_id,ip,name,start_date,end_date
date,,,,,,,


In [60]:
data_science.groupby(['name','user_id']).page_viewed.value_counts()

name    user_id  page_viewed                                      
Bayes   358      search/search_index.json                             63
                 6-regression/1-overview                              28
                 10-anomaly-detection/1-overview                      22
                 10-anomaly-detection/AnomalyDetectionCartoon.jpeg    22
                 5-stats/3-probability-distributions                  19
                                                                      ..
Darden  785      sql/mysql-overview                                    1
                 timeseries/modeling-lesson1                           1
                 timeseries/prep                                       1
                 timeseries/project                                    1
                 timeseries/working-with-time-series-data              1
Name: page_viewed, Length: 10383, dtype: int64

# 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?


In [61]:
bayes = data_science[data_science.name == 'Bayes']

In [79]:
bayes.page_viewed.value_counts().head(20)

/                                                    1842
1-fundamentals/modern-data-scientist.jpg              626
1-fundamentals/AI-ML-DL-timeline.jpg                  624
1-fundamentals/1.1-intro-to-data-science              615
search/search_index.json                              551
6-regression/1-overview                               521
10-anomaly-detection/AnomalyDetectionCartoon.jpeg     386
10-anomaly-detection/1-overview                       383
6-regression/5.0-evaluate                             333
5-stats/3-probability-distributions                   320
5-stats/4.2-compare-means                             315
appendix/cli-git-overview                             311
6-regression/7.0-model                                310
6-regression/4.0-explore                              267
6-regression/3.0-split-and-scale                      260
7-classification/3-prep                               256
4-python/7.4.3-dataframes                             251
7-classificati

In [71]:
darden = data_science[data_science.name == 'Darden']

In [75]:
darden.page_viewed.value_counts().head(20)

/                                           2041
classification/overview                      759
classification/scale_features_or_not.svg     590
sql/mysql-overview                           513
1-fundamentals/modern-data-scientist.jpg     470
1-fundamentals/AI-ML-DL-timeline.jpg         470
1-fundamentals/1.1-intro-to-data-science     460
stats/compare-means                          338
classification/logistic-regression           334
classification/prep                          321
search/search_index.json                     300
1-fundamentals/DataToAction_v2.jpg           284
classification/explore                       282
classification/evaluation                    280
1-fundamentals/1.2-data-science-pipeline     271
classification/project                       252
classification/acquire                       252
stats/probability-distributions              246
python/data-types-and-variables              235
stats/correlation                            234
Name: page_viewed, d

In [76]:
curie = data_science[data_science.name == 'Curie']

In [77]:
curie.page_viewed.value_counts().head(20)

/                                                    1523
6-regression/1-overview                               595
search/search_index.json                              480
1-fundamentals/modern-data-scientist.jpg              467
1-fundamentals/AI-ML-DL-timeline.jpg                  465
1-fundamentals/1.1-intro-to-data-science              461
3-sql/1-mysql-overview                                441
10-anomaly-detection/AnomalyDetectionCartoon.jpeg     345
10-anomaly-detection/1-overview                       345
4-python/8.4.3-dataframes                             260
4-python/8.4.4-advanced-dataframes                    246
4-python/3-data-types-and-variables                   234
4-python/5-functions                                  203
5-stats/4.2-compare-means                             197
5-stats/2-simulation                                  193
appendix/cli-git-overview                             190
3-sql/7-functions                                     185
7-classificati

In [80]:
bayes.head()

,page_viewed,user_id,cohort_id,ip,name,start_date,end_date
date,,,,,,,
2019-08-20 09:39:58,/,466,34,97.105.19.58,Bayes,2019-08-19,2020-01-30
2019-08-20 09:39:59,/,467,34,97.105.19.58,Bayes,2019-08-19,2020-01-30
2019-08-20 09:39:59,/,468,34,97.105.19.58,Bayes,2019-08-19,2020-01-30
2019-08-20 09:40:02,/,469,34,97.105.19.58,Bayes,2019-08-19,2020-01-30
2019-08-20 09:40:08,/,470,34,97.105.19.58,Bayes,2019-08-19,2020-01-30


In [98]:
web_dev.name.value_counts()

Staff         60315
Ceres         40168
Zion          37548
Fortuna       36047
Voyageurs     35624
Ganymede      32447
Apex          31670
Teddy         30328
Deimos        29688
Hyperion      28354
Betelgeuse    28333
Ulysses       28329
Xanadu        27637
Europa        27518
Wrangell      25427
Andromeda     25253
Yosemite      20447
Bash          11774
Lassen         9495
Arches         8884
Jupiter        8718
Sequoia        7444
Olympic        4925
Kings          2733
Pinnacles      2005
Hampton        1712
Quincy         1165
Niagara         743
Mammoth         691
Glacier         495
Joshua          302
Ike             208
Badlands         93
Franklin         72
Apollo            5
Denali            4
Everglades        1
Name: name, dtype: int64

In [96]:
ceres = web_dev[web_dev.name == 'Ceres']

In [97]:
ceres.page_viewed.value_counts().head(20)

/                                                                            1620
search/search_index.json                                                     1376
javascript-i                                                                  977
toc                                                                           909
html-css                                                                      753
java-iii                                                                      674
java-ii                                                                       667
jquery                                                                        632
mysql                                                                         617
spring                                                                        546
javascript-ii                                                                 519
java-i                                                                        510
html-css/css-i/f

In [99]:
zion = web_dev[web_dev.name == 'Zion']

In [101]:
zion.page_viewed.value_counts().head(20)

/                                                                            1756
toc                                                                          1457
javascript-i                                                                  868
java-iii                                                                      742
search/search_index.json                                                      689
spring                                                                        650
html-css                                                                      649
javascript-ii                                                                 637
java-ii                                                                       613
mysql                                                                         598
java-i                                                                        593
jquery                                                                        559
spring/fundament

In [102]:
fortuna = web_dev[web_dev.name == 'Fortuna']

In [103]:
fortuna.page_viewed.value_counts().head(20)

/                                    1962
toc                                  1273
search/search_index.json              989
java-iii                              767
javascript-i                          756
java-ii                               637
spring                                616
html-css                              578
mysql                                 571
java-i                                538
jquery                                501
javascript-ii                         482
java-iii/servlets                     416
java-iii/jsp-and-jstl                 402
mysql/tables                          373
java-i/syntax-types-and-variables     358
java-i/introduction-to-java           356
mysql/basic-statements                348
spring/fundamentals/controllers       343
appendix                              337
Name: page_viewed, dtype: int64

In [104]:
voyageurs = web_dev[web_dev.name == 'Voyageurs']

In [106]:
voyageurs.page_viewed.value_counts().head(10)

/                2098
javascript-i      884
java-iii          770
java-ii           754
mysql             663
spring            650
java-i            639
javascript-ii     584
jquery            583
html-css          528
Name: page_viewed, dtype: int64

In [107]:
ganymede = web_dev[web_dev.name == 'Ganymede']

In [108]:
ganymede.page_viewed.value_counts().head(10)

/                           1618
search/search_index.json    1050
toc                          810
javascript-i                 694
java-iii                     620
java-ii                      607
appendix                     526
jquery                       496
javascript-ii                483
java-i                       482
Name: page_viewed, dtype: int64

**for data science it looks like fundamentals is the most common. For web dev it looks like java and javascript are the most common.**